In [2]:
import pandas as pd
# pd.set_option('display.max_rows', 100) #设置为80
# pd.set_option('display.max_colwidth', 1500) #设置为80

from tqdm import tqdm
tqdm.pandas() 

import sys
sys.path.append('../')
from tool_llm_response.call_llm_with_zny import CallLLMByZny,ZnyConfig
from tool_kg_search.get_1b_output import get_thought_api
from tool_kg_search.get_api_obs import get_api_df, get_obs_df
from tool_kg_search.search_http_tool import SearchByHttpTool
from tool_kg_search.kgsearch_llm_obs import *
from tool_rag_generation.data_format import DataFormat

### 样例数据

In [12]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'thought':['涉及事实问答，查询世界第三高峰的信息。'],
'api':[[{"category":"地理","query":"世界第三高峰是哪个","tag":"世界第三高峰","name":"qasearch"}]],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)
df

,id,turn_id,source,user-query,Thought,API,observation,assistant,relevant_label
0,test-1,1,测试,世界的第三高峰是哪个,涉及事实问答，查询世界第三高峰的信息。,"[{'category': '地理', 'query': '世界第三高峰是哪个', 'tag': '世界第三高峰', 'name': 'qasearch'}]","[['一些内容','一些内容','一些内容']]",世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。,相关


### 单独调用API

In [6]:
data = {'user-query': ['介绍下周杰伦', '今天有什么新闻', '介绍下理想L9'], 'category': ['qaearch', 'mediasearch', 'autosearch']}
df = pd.DataFrame(data)
url = 'http://172.24.139.202:16073/ligpt_with_api/search' # 服务不可用时，联系 辛洪生 获取最新url

df[['thought', 'api']] = df.progress_apply(lambda row: get_thought_api(row['user-query'], row['category'], url), axis=1, result_type='expand')
df

100%|██████████| 3/3 [00:02<00:00,  1.07it/s]


,user-query,category,Thought,API
0,介绍下周杰伦,qaearch,[用户请求介绍周杰伦，涉及人物知识，查询周杰伦的介绍信息],"[{'category': '人物', 'query': '周杰伦的介绍', 'tag': '周杰伦&介绍', 'name': 'qasearch'}]"
1,今天有什么新闻,mediasearch,[今天有什么新闻],"[{'category': '新闻', 'query': '今天有什么新闻', 'publishtime': '今天', 'name': 'qasearch'}]"
2,介绍下理想L9,autosearch,[涉及汽车问答，查询理想L9的简介信息做参考],"[{'category': '汽车', 'query': '理想L9的简介', 'tag': '理想L9&简介', 'name': 'autosearch'}]"


### 单独调用搜索

In [2]:
# 与上文API无关
search_env = 'app_dev'     #调用的搜索环境，取值: arch | dev | liping | das | faq ... 
control_param =  {      # 搜索需要的参数, 默认都不打开
    # "disable-bing-cache":"true",      # 是否不要bing cache
}
search_http_tool = SearchByHttpTool(search_env=search_env, 
                                    limit=-1,       # -1 则采用搜索内部默认, >0则手动指定 
                                    control_param=control_param)

# query = '今天有什么新闻'
# input_api = [{'APINAME': 'MEDIASearch',
#   'MEDIATYPE': '新闻',
#   'QUERY': '今天有什么新闻',
#   'PUBLISHTIME': '今天',
#   'STARTTIME': '{"norm":{"day":"+0"},"raw":"今天"}',
#   "timeSlots": '{"type": "ABSOLUTE_TIME","range": "POINT","format": "yyyy-MM-dd HH:mm:ss","start": {"raw": "2024年","time": "2024-07-02 00:00:00","norm": {"year": "2024"}},"supplemental": {"time": "2024-07-02 23:59:59"}}'}]

query = '周杰伦和王力宏谁厉害'
input_api = [{"apiname": "QASearch", "category": "通用问答", "query": "周杰伦的简介", "tag": "周杰伦&简介"},
             {"apiname": "QASearch", "category": "通用问答", "query": "王力宏的简介", "tag": "王力宏&简介"}]

# query = '目前最先进的ai芯片是哪家公司的'
# input_api = [{"STARTTIME":"{\"norm\":{\"now\":\"true\"},\"raw\":\"目前\"}","APINAME":"QASearch","QUERY":"目前最先进的AI芯片是哪家公司的","CATEGORY":"公司","TAG":"目前&最先进&AI芯片&公司&时效性","timeslots":"{\"type\":\"ABSOLUTE_TIME\",\"range\":\"POINT\",\"format\":\"yyyy-MM-dd HH:mm:ss\",\"start\":{\"raw\":\"目前\",\"time\":\"2024-05-24 21:01:56\",\"norm\":{\"now\":\"true\"}},\"end\":null,\"duration\":null,\"supplemental\":{\"raw\":\"\",\"time\":\"2024-05-24 21:01:56\",\"norm\":null},\"interval\":\"\"}"}]
multi_llm_obs = get_obs(search_http_tool, query, input_api)
multi_llm_obs

[[{'id': 'd7a7ca1afe07090a62be2181aea00334',
   'rel_score': 0.9915447,
   'title': '周杰伦简介',
   'source_link': 'https://baike.baidu.com/item/%E5%91%A8%E6%9D%B0%E4%BC%A6/129156?fromtitle=%C3%A5%C2%91%C2%A8%C3%A8%C2%91%C2%A3&fromid=486125',
   'content': '周杰伦简介 周杰伦（Jay Chou），1979年1月18日出生于台湾省新北市，祖籍福建省泉州市永春县，华语流行乐男歌手、音乐人、演员、导演、编剧，毕业于淡江中学。\n2000年，发行个人首张音乐专辑《Jay》。2001年，凭借专辑《范特西》奠定其融合中西方音乐的风格。2002年，举行“The One”世界巡回演唱会。2003年，成为美国《时代周刊》封面人物；同年，发行音乐专辑《叶惠美》，该专辑获得第15届台湾金曲奖最佳流行音乐演唱专辑奖。2004年，发行音乐专辑《七里香》，该专辑在全亚洲的首月销量达到300万张；同年，获得世界音乐大奖中国区最畅销艺人奖。2005年，主演个人首部电影《头文字D》，并凭借该片获得第25届香港电影金像奖和第42届台湾电影金马奖的最佳新演员奖。2006年起，连续三年获得世界音乐大奖中国区最畅销艺人奖。\n2007年，自编自导爱情电影《不能说的秘密》；同年，成立杰威尔音乐有限公司。2008年，凭借歌曲《青花瓷》获得第19届台湾金曲奖最佳作曲人奖。2009年，入选美国CNN“25位亚洲最具影响力人物”；同年，凭借专辑《魔杰座》获得第20届台湾金曲奖最佳国语男歌手奖。2010年，入选美国《Fast Company》评出的“全球百大创意人物”。2011年，凭借专辑《跨时代》获得第22届台湾金曲奖最佳国语男歌手奖。2012年，登上福布斯中国名人榜榜首。2014年，发行个人首张数字音乐专辑《哎呦，不错哦》。2023年，凭借专辑《最伟大的作品》成为首位获得国际唱片业协会“全球畅销专辑榜”冠军的华语歌手。',
   'paragraph_id': 'd7a7ca1afe07090a62be2181aea00334',
   'doc_id': '67691

### 调用其他大模型蒸馏

In [10]:
def get_prompts_df(df: pd.DataFrame, oneshot_prompt:str) -> pd.DataFrame:
    df['prompts'] = df.apply(lambda row: oneshot_prompt + f"""
    ---
    下面是给出的实际问题：
    Question:
    {row['query']}
    """, axis=1)
    
    return df

In [7]:
folder = '/mnt/pfs-guan-ssai/nlu/baoyuyang/sft_open_data/SFT数据集质量检测_5k/gpt4-instruct-answer真实性/'
df = pd.read_csv(folder+'sample_authentic_res.csv_0.csv')[:5]
df

,unique_id,id,messages,target_value,source,system_prompt,last_query,last_assistant,owner,file_name,...,label2,create_user,update_user,create_time,update_time,data_version,query,ans,ans真实性检测结果,真实性score
0,1870397,NaN,"[{'role': 'user', 'content': '这是一个表格：\n| 基尼指数 ...",这是一个表格：\n| 基尼指数 | 定义 | 描述 |\n| --- | --- | ---...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,NaN,baoyuyang,baoyuyang,NaN,NaN,NaN,这是一个表格：\n| 基尼指数 | 定义 | 描述 |\n| --- | --- | ---...,基尼指数的计算根据给定数据集$A$，首先需要计算在$A$上的基尼指数$Gini(A)$，使用...,原子信息：\n1. 基尼指数的计算首先需要计算在数据集$A$上的基尼指数$Gini(A)$。...,1.00
1,1785692,NaN,"[{'role': 'user', 'content': '奥利的香蕉船上有x勺冰淇淋，而维...",奥利的香蕉船上有x勺冰淇淋，而维多利亚的冰淇淋比奥利的多两倍。维多利亚比奥利多4勺冰淇淋。未...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,NaN,baoyuyang,baoyuyang,NaN,NaN,NaN,奥利的香蕉船上有x勺冰淇淋，而维多利亚的冰淇淋比奥利的多两倍。维多利亚比奥利多4勺冰淇淋。未...,为了解决这个问题，我们需要确定代表奥利的香蕉船中冰淇淋勺数的x的值。我们知道，维多利亚的冰淇...,原子信息：\n1. 奥利的香蕉船上有x勺冰淇淋（这是题目的设定，正确，打【1】分）\n2. ...,1.00
2,1065670,NaN,"[{'role': 'user', 'content': '给定一段文本和一个主题，将文本中...",给定一段文本和一个主题，将文本中的所有有关该主题的句子提取出来。\n主题: 员工培训\n文本...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,NaN,baoyuyang,baoyuyang,NaN,NaN,NaN,给定一段文本和一个主题，将文本中的所有有关该主题的句子提取出来。\n主题: 员工培训\n文本...,与主题“员工培训”相关的句子：\n- 训练可以使员工掌握新技能、提高绩效和生产力，提高员工的...,原子信息：\n1. 训练可以使员工掌握新技能（正确，打【1】分）\n2. 训练可以提高绩效和...,1.00
3,2156990,NaN,"[{'role': 'user', 'content': '说明人类大脑的基本结构和功能，包...",说明人类大脑的基本结构和功能，包括大脑分区和其作用。 人类大脑由左右两个半球组成，大脑分为皮...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,NaN,baoyuyang,baoyuyang,NaN,NaN,NaN,说明人类大脑的基本结构和功能，包括大脑分区和其作用。,人类大脑由左右两个半球组成，大脑分为皮层和脑深部结构。皮层是大脑的最外层，控制感官、思维和运...,原子信息：\n1. 人类大脑由左右两个半球组成。（正确，打【1】分）\n2. 大脑分为皮层和...,0.75
4,2895694,NaN,"[{'role': 'user', 'content': '请分析下面的XML数据，其中包含...",请分析下面的XML数据，其中包含了调查结果，你认为是什么具体因素导致了社区对限制河流可达性的...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,NaN,baoyuyang,baoyuyang,NaN,NaN,NaN,请分析下面的XML数据，其中包含了调查结果，你认为是什么具体因素导致了社区对限制河流可达性的...,根据调查结果，社区对限制河流可达性的新规定存在正反两种不同意见，因此需要寻找一种平衡的替代方...,原子信息：\n 1. 社区对限制河流可达性的新规定存在正反两种不同意见（正确，打【1...,1.00


In [13]:
config = ZnyConfig(
    url = 'https://rhm-gpt4.fc.chj.cloud/gpt4o', # 智能云GPT api
    # url = 'https://baoyuyang-sftdataset-test1.fc.chj.cloud',
    model_name = 'gpt4o',
    temperature = 0.5, # llm输出温度，zny下的gpt4基本无效，因为是全球节点，还是会有随机性
    max_retries = 5, # 调用gpt报错后最多重试 max_retries 次
    qps = 5, # 多线程 or 异步多线程下，qps，不要超过5
    max_concurrent = 10, # 异步多线程参数，一般10或者20，太大会接口超过qps
    asyncio_flag = False, # True=异步多线程，只能python调用；False=普通多线程，Jupyter或者python均可
    query_column_name = 'prompts', # llm模型输入列名
    response_column_name = 'gpt4response' # llm模型输出列名
)

call_zny = CallLLMByZny(config)

oneshot_prompt ='根据参考资料回答问题：'
prompt_df = get_prompts_df(df, oneshot_prompt)
prompt_df

,unique_id,id,messages,target_value,source,system_prompt,last_query,last_assistant,owner,file_name,...,create_user,update_user,create_time,update_time,data_version,query,ans,ans真实性检测结果,真实性score,prompts
0,1870397,NaN,"[{'role': 'user', 'content': '这是一个表格：\n| 基尼指数 ...",这是一个表格：\n| 基尼指数 | 定义 | 描述 |\n| --- | --- | ---...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,baoyuyang,NaN,NaN,NaN,这是一个表格：\n| 基尼指数 | 定义 | 描述 |\n| --- | --- | ---...,基尼指数的计算根据给定数据集$A$，首先需要计算在$A$上的基尼指数$Gini(A)$，使用...,原子信息：\n1. 基尼指数的计算首先需要计算在数据集$A$上的基尼指数$Gini(A)$。...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...
1,1785692,NaN,"[{'role': 'user', 'content': '奥利的香蕉船上有x勺冰淇淋，而维...",奥利的香蕉船上有x勺冰淇淋，而维多利亚的冰淇淋比奥利的多两倍。维多利亚比奥利多4勺冰淇淋。未...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,baoyuyang,NaN,NaN,NaN,奥利的香蕉船上有x勺冰淇淋，而维多利亚的冰淇淋比奥利的多两倍。维多利亚比奥利多4勺冰淇淋。未...,为了解决这个问题，我们需要确定代表奥利的香蕉船中冰淇淋勺数的x的值。我们知道，维多利亚的冰淇...,原子信息：\n1. 奥利的香蕉船上有x勺冰淇淋（这是题目的设定，正确，打【1】分）\n2. ...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...
2,1065670,NaN,"[{'role': 'user', 'content': '给定一段文本和一个主题，将文本中...",给定一段文本和一个主题，将文本中的所有有关该主题的句子提取出来。\n主题: 员工培训\n文本...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,baoyuyang,NaN,NaN,NaN,给定一段文本和一个主题，将文本中的所有有关该主题的句子提取出来。\n主题: 员工培训\n文本...,与主题“员工培训”相关的句子：\n- 训练可以使员工掌握新技能、提高绩效和生产力，提高员工的...,原子信息：\n1. 训练可以使员工掌握新技能（正确，打【1】分）\n2. 训练可以提高绩效和...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...
3,2156990,NaN,"[{'role': 'user', 'content': '说明人类大脑的基本结构和功能，包...",说明人类大脑的基本结构和功能，包括大脑分区和其作用。 人类大脑由左右两个半球组成，大脑分为皮...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,baoyuyang,NaN,NaN,NaN,说明人类大脑的基本结构和功能，包括大脑分区和其作用。,人类大脑由左右两个半球组成，大脑分为皮层和脑深部结构。皮层是大脑的最外层，控制感官、思维和运...,原子信息：\n1. 人类大脑由左右两个半球组成。（正确，打【1】分）\n2. 大脑分为皮层和...,0.75,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...
4,2895694,NaN,"[{'role': 'user', 'content': '请分析下面的XML数据，其中包含...",请分析下面的XML数据，其中包含了调查结果，你认为是什么具体因素导致了社区对限制河流可达性的...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,baoyuyang,NaN,NaN,NaN,请分析下面的XML数据，其中包含了调查结果，你认为是什么具体因素导致了社区对限制河流可达性的...,根据调查结果，社区对限制河流可达性的新规定存在正反两种不同意见，因此需要寻找一种平衡的替代方...,原子信息：\n 1. 社区对限制河流可达性的新规定存在正反两种不同意见（正确，打【1...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...


In [14]:
gpt4_df = call_zny.get_gpt4api_df(prompt_df)
gpt4_df

剩余case 5 / 5


  0%|          | 0/5 [00:00<?, ?it/s]

chatgpt api 调用异常：Expecting value: line 1 column 1 (char 0)


100%|██████████| 5/5 [00:16<00:00,  3.28s/it]


,unique_id,id,messages,target_value,source,system_prompt,last_query,last_assistant,owner,file_name,...,update_user,create_time,update_time,data_version,query,ans,ans真实性检测结果,真实性score,prompts,gpt4response
0,1870397,NaN,"[{'role': 'user', 'content': '这是一个表格：\n| 基尼指数 ...",这是一个表格：\n| 基尼指数 | 定义 | 描述 |\n| --- | --- | ---...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,NaN,NaN,NaN,这是一个表格：\n| 基尼指数 | 定义 | 描述 |\n| --- | --- | ---...,基尼指数的计算根据给定数据集$A$，首先需要计算在$A$上的基尼指数$Gini(A)$，使用...,原子信息：\n1. 基尼指数的计算首先需要计算在数据集$A$上的基尼指数$Gini(A)$。...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...,基尼指数用于度量数据集中的不平等度，可以通过以下公式进行计算：\n\n1. **基尼指数 \...
1,1785692,NaN,"[{'role': 'user', 'content': '奥利的香蕉船上有x勺冰淇淋，而维...",奥利的香蕉船上有x勺冰淇淋，而维多利亚的冰淇淋比奥利的多两倍。维多利亚比奥利多4勺冰淇淋。未...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,NaN,NaN,NaN,奥利的香蕉船上有x勺冰淇淋，而维多利亚的冰淇淋比奥利的多两倍。维多利亚比奥利多4勺冰淇淋。未...,为了解决这个问题，我们需要确定代表奥利的香蕉船中冰淇淋勺数的x的值。我们知道，维多利亚的冰淇...,原子信息：\n1. 奥利的香蕉船上有x勺冰淇淋（这是题目的设定，正确，打【1】分）\n2. ...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...,根据题意，我们可以设置以下方程来求解未知数x：\n\n1. **设奥利的冰淇淋勺数为x**。...
2,1065670,NaN,"[{'role': 'user', 'content': '给定一段文本和一个主题，将文本中...",给定一段文本和一个主题，将文本中的所有有关该主题的句子提取出来。\n主题: 员工培训\n文本...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,NaN,NaN,NaN,给定一段文本和一个主题，将文本中的所有有关该主题的句子提取出来。\n主题: 员工培训\n文本...,与主题“员工培训”相关的句子：\n- 训练可以使员工掌握新技能、提高绩效和生产力，提高员工的...,原子信息：\n1. 训练可以使员工掌握新技能（正确，打【1】分）\n2. 训练可以提高绩效和...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...,根据给定的主题“员工培训”，以下是从文本中提取出的有关该主题的句子：\n\n1. 训练是组织...
3,2156990,NaN,"[{'role': 'user', 'content': '说明人类大脑的基本结构和功能，包...",说明人类大脑的基本结构和功能，包括大脑分区和其作用。 人类大脑由左右两个半球组成，大脑分为皮...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,NaN,NaN,NaN,说明人类大脑的基本结构和功能，包括大脑分区和其作用。,人类大脑由左右两个半球组成，大脑分为皮层和脑深部结构。皮层是大脑的最外层，控制感官、思维和运...,原子信息：\n1. 人类大脑由左右两个半球组成。（正确，打【1】分）\n2. 大脑分为皮层和...,0.75,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...,人类大脑是一个复杂而高度组织的器官，负责控制和协调身体的各种功能。大脑的基本结构和功能可以通...
4,2895694,NaN,"[{'role': 'user', 'content': '请分析下面的XML数据，其中包含...",请分析下面的XML数据，其中包含了调查结果，你认为是什么具体因素导致了社区对限制河流可达性的...,Subjective,NaN,NaN,NaN,baoyuyang,NaN,...,baoyuyang,NaN,NaN,NaN,请分析下面的XML数据，其中包含了调查结果，你认为是什么具体因素导致了社区对限制河流可达性的...,根据调查结果，社区对限制河流可达性的新规定存在正反两种不同意见，因此需要寻找一种平衡的替代方...,原子信息：\n 1. 社区对限制河流可达性的新规定存在正反两种不同意见（正确，打【1...,1.00,根据参考资料回答问题：\n ---\n 下面是给出的实际问题：\n Que...,根据提供的XML数据，社区对限制河流可达性的新规定存在不同意见，支持和反对的比例各占一半。分...


### 生成训练数据格式（中间状态，jsonl文件）

In [10]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'Thought':['涉及事实问答，查询世界第三高峰的信息。'],
'API':[[{"category":"地理","query":"世界第三高峰是哪个","tag":"世界第三高峰","name":"qasearch"}]],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)
df = df.astype(str)
df

,id,turn_id,source,user-query,Thought,API,observation,assistant,relevant_label
0,test-1,1,测试,世界的第三高峰是哪个,涉及事实问答，查询世界第三高峰的信息。,"[{'category': '地理', 'query': '世界第三高峰是哪个', 'tag': '世界第三高峰', 'name': 'qasearch'}]","[['一些内容','一些内容','一些内容']]",世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。,相关


In [11]:
dataformat_obj = DataFormat(api_flag=True,multi_flag=False)
sft_df = dataformat_obj.gen_sft_data(df)
sft_df

sft 数量： 1


,id,turn_id,source,messages,system
0,test-1,1,测试,"[{'role': 'user', 'content': ['世界的第三高峰是哪个']}, {'role': 'thought', 'content': ['涉及事实问答，查询世界第三高峰的信息。']}, {'role': 'api', 'content': [{'token': '<|api_start|>'}, {'token': '<|kvs|>'}, 'category', {'token': '=>'}, '地理', {'token': '<|kve|>'}, {'token': '<|kvs|>'}, 'query', {'token': '=>'}, '世界第三高峰是哪个', {'token': '<|kve|>'}, {'token': '<|kvs|>'}, 'tag', {'token': '=>'}, '世界第三高峰', {'token': '<|kve|>'}, {'token': '<|kvs|>'}, 'name', {'token': '=>'}, 'qasearch', {'token': '<|kve|>'}, {'token': '<|api_end|>'}]}, {'role': 'observation', 'content': [{'token': '<|kvs|>'}, '{""QASearchResults"": [""一些内容"", ""一些内容"", ""一些内容""]}', {'token': '<|kve|>'}]}, {'role': 'assistant', 'content': ['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。']}]",[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智能空间部门创造。\n理想同学能够理解人类的指令和意图，并且给出合理的、切合问题的、没有歧视、中立的、安全的回复。\n\n请根据以下文本写一个合适的回复。[unused1]\n


### 生成模型instruction格式

In [3]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'Thought':['涉及事实问答，查询世界第三高峰的信息。'],
'API':[[{"category":"地理","query":"世界第三高峰是哪个","tag":"世界第三高峰","name":"qasearch"}]],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)
df = df.astype(str)

In [4]:
dataformat_obj = DataFormat(api_flag=True,multi_flag=False) 
# api_flag: True rag 格式，thought api observation字段必填； False thought api observation字段非必填
# multi_flag：True 需要有context字段，False 不需要有context字段
sft_data = dataformat_obj.gen_sft_unused_data(df)
sft_df = pd.DataFrame(sft_data)
sft_df

,instruction,output
0,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。[unused1]


In [5]:
sft_df.iloc[0]['instruction']

'[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智能空间部门创造。\n理想同学能够理解人类的指令和意图，并且给出合理的、切合问题的、没有歧视、中立的、安全的回复。\n\n请根据以下文本写一个合适的回复。[unused1]\n[unused0]user\n世界的第三高峰是哪个[unused1]\n[unused0]thought\n涉及事实问答，查询世界第三高峰的信息。[unused1]\n[unused0]api\n[unused4][unused2]category[unused7]地理[unused3][unused2]query[unused7]世界第三高峰是哪个[unused3][unused2]tag[unused7]世界第三高峰[unused3][unused2]name[unused7]qasearch[unused3][unused5][unused6][unused1]\n[unused0]observation\n[unused2]{"qasearchResults": ["一些内容", "一些内容", "一些内容"]}[unused3][unused1]\n[unused0]assistant\n'

### query 到 model_13b_input格式

In [9]:
query = ['今天有什么新闻']
df = pd.DataFrame(query,columns=['user-query'])

# 调用api
df_api = get_api_df(df, col_query='user-query', output_file='./api.csv', 
                    url='http://172.24.139.202:16073/ligpt_with_api/search')

# 调用obs
df_obs = get_obs_df(df_api, length_limit=20000, output_path='./obs.csv') #length_limit 表示obs最大长度，超长pop

# 转model_13b_input格式
dataformat_obj = DataFormat(api_flag=True,multi_flag=False) 
sft_data = dataformat_obj.gen_sft_unused_data(df_obs)
sft_df = pd.DataFrame(sft_data)
sft_df


100%|██████████| 1/1 [00:00<00:00,  4.96it/s]


,instruction,output
0,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,无[unused1]
